# Physics Informed Neural Network



In [47]:
# Importing packages
using Flux
using Plots
using Statistics


In [43]:
# double differentiation

# f will become our solution for x in [0,1]
f=Chain(x->[x],Dense(1, 10, tanh),Dense(10, 10, tanh),Dense(10, 1),first)
p=Flux.params(f)

# create samples for x in [0,1]
n=10
x_samples=Flux.rand32(n)

# testing the function f
# y_samples=f.(x_samples)
# y_samples

# define the function f_x(x)=df/dx
f_x(x)=gradient(f,x)[1]

# define our ODE df/dx=-x
equation(x)=f_x(x)+1f0*x

# Loss is L2 norm of residual (on samples)
function loss(p)
    residual=equation.(x_samples)
    return mean(residual.^2)
end

val=loss(p)
val

# try gradient of loss
(val,grad)=Flux.withgradient(loss,p) # QUESTION: Zygote does not like loss function?
val,grad

#CONCLUSION: I still do not know how to use Flux to implement a PINN. 


MethodError: MethodError: no method matching loss()

Closest candidates are:
  loss(!Matched::Any)
   @ Main ~/src_nobackup/julia_ml_tests.jl.git/flux_pinn.ipynb:22


In [68]:
# double differentiation

# f will become our solution for x in [0,1]
f=Chain(Dense(1, 10, tanh),Dense(10, 10, tanh),Dense(10, 1))
p=Flux.params(f)

# create samples for x in [0,1]
n=10
x_samples=Flux.rand32(1,n) # Flux wants the samples as a final dimension

# testing the function f
y_samples=f(x_samples)
y_samples

# define the function f_x(x)=df/dx
# The sum is a trick to compute the scalar derivatives of a vector function
#dy = gradient(x->sum(f(x)), x_samples)
#dy
f_x(x)=first(gradient(x->sum(f(x)),x))
# dy=f_x(x_samples)
# dy

# # define our ODE df/dx=-x
equation(x)=f_x(x).+1f0.*x
# res=equation(x_samples)
# res

# Loss is L2 norm of residual (on samples)
function loss(p)
    residuals=equation(x_samples)
    return sum(residuals.^2) # QUESTION: abs2 and Flux.mse give errors
end

# val=loss(p)
# val

# # try gradient of loss
grad=Flux.gradient(loss,p) # QUESTION: Flux.gradient gives error
val,grad

#CONCLUSION: I still do not know how to use Flux to implement a PINN. 


MethodError: MethodError: no method matching loss()

Closest candidates are:
  loss(!Matched::Any)
   @ Main ~/src_nobackup/julia_ml_tests.jl.git/flux_pinn.ipynb:29


In [ ]:
# I also asked ChatGPT. It made the same mistakes, but then gave a good suggestion, ie to use the SUM.
# This is a nice trick to work with all values at once, instead of one by one.

function loss(x)
    # Predicted function values
    f̂ = model(x)
    
    # Derivative of the predicted function (gradient wrt x)
    df̂ = gradient(x_ -> sum(model(x_)), x)[1]
    
    # Enforcing f'(x) + x = 0
    eq_loss = mean((df̂ .+ x).^2)
    
    return eq_loss
end
